## Example: Observing Convergence in a Two-Room Markov Chain with Random Probabilities


This Python code demonstrates how to:

1. **Randomly generate** probabilities for the independent object's movements ($T_I$)
   and the dependent object's attachment ($a_0, a_1$).
2. Construct a **block-diagonal** $8 \times 8$ transition matrix $\mathbf{P}$
   reflecting two rooms and three objects:
   - A **static** object (never moves),
   - An **independent** object (moves according to $T_I$),
   - A **dependent** object (can “attach” to the independent object with probability
     $a_r$).
3. Create a **random initial distribution** $\mathbf{p}(0)$ by sampling the
   probability of each object being in room 0 vs. room 1, then normalizing to ensure it
   sums to 1.
4. Perform an iterative **convergence check** by repeatedly applying $\mathbf{p}(t+1) =
   \mathbf{P}^\top \mathbf{p}(t)$ and measuring the **L1 distance** $\|\mathbf{p}(t+1) -
   \mathbf{p}(t)\|_{1}$. If this distance drops below a chosen tolerance (e.g.,
   $10^{-3}$), the distribution is deemed converged.
5. Log:
   - The first few steps of convergence to see how the distribution evolves,
   - The step at which the convergence condition is reached,
   - The final converged distribution (or a message if it did not converge within
     `max_steps`).

### Key Points

- **`T_I_random`**: For each row (0 and 1), we draw two random numbers and normalize so
  each row sums to 1. This yields valid probabilities for the independent object moving
  or staying in the same room.
- **`a_0_random`, `a_1_random`**: Each is a random float in $[0,1]$. They control how
  likely the dependent object will “attach” when co-located in rooms 0 or 1,
  respectively.
- **Block-diagonal structure**:
  - The static object’s location $s$ splits the state space into **two disconnected
    4-state blocks** (for $s=0$ and $s=1$).
  - Each block captures the transitions of $(i, d)$ given $s$.
- **Random initial distribution** $\mathbf{p}(0)$:
  - Probability that each object is in room 0 is drawn at random,
  - Then combined (multiplied) for joint states $(s,i,d)$,
  - Finally normalized in case of floating-point anomalies.
- **Convergence measure**:
  - We track `step_needed_for_convergence.append(step)` for each random seed in a list,
    showing how quickly different random configurations of $\mathbf{P}$ and
    $\mathbf{p}(0)$ converge.
  - Increasing `tolerance` or reducing `max_steps` can speed up or slow convergence
    checks, at the risk of less or more precise final distributions.

This loop runs **10 different seeds**, resulting in 10 sets of random probabilities and
potentially 10 different numbers of steps needed to converge. You can easily compare how
quickly each configuration converges by printing the values in
`step_needed_for_convergence` after the loop finishes.


In [1]:
import numpy as np


step_needed_for_convergence = []
for seed in range(10):
    # Random seed (optional) for reproducibility
    np.random.seed(seed)

    # --------------------------------------------------------------------
    # 1. Randomly define movement probabilities T_I for the independent object
    # --------------------------------------------------------------------
    # We'll generate two rows of random numbers and normalize each row so
    # they sum to 1. That way, T_I[i_from] is a valid probability distribution.
    T_I_random = np.random.rand(2, 2)
    for i in range(2):
        T_I_random[i] /= T_I_random[i].sum()

    print("Random T_I:\n", T_I_random)

    # --------------------------------------------------------------------
    # 2. Randomly define attachment probabilities a_0, a_1 in [0,1]
    # --------------------------------------------------------------------
    a_0_random = np.random.rand()
    a_1_random = np.random.rand()

    print("\nRandom attachment probabilities: a_0 =", a_0_random, ", a_1 =", a_1_random)

    # --------------------------------------------------------------------
    # 3. Build the 4x4 blocks P_s0, P_s1 with these random probabilities
    # --------------------------------------------------------------------
    P_s0_random = np.zeros((4, 4))

    # (0,0,0) -> row 0
    P_s0_random[0, 0] = T_I_random[0, 0]
    P_s0_random[0, 2] = T_I_random[0, 1] * (1 - a_0_random)
    P_s0_random[0, 3] = T_I_random[0, 1] * a_0_random

    # (0,0,1) -> row 1
    P_s0_random[1, 1] = T_I_random[0, 0]
    P_s0_random[1, 3] = T_I_random[0, 1]

    # (0,1,0) -> row 2
    P_s0_random[2, 0] = T_I_random[1, 0]
    P_s0_random[2, 2] = T_I_random[1, 1]

    # (0,1,1) -> row 3
    P_s0_random[3, 0] = T_I_random[1, 0] * a_1_random
    P_s0_random[3, 1] = T_I_random[1, 0] * (1 - a_1_random)
    P_s0_random[3, 3] = T_I_random[1, 1]

    # Static object s=1 => identical block
    P_s1_random = P_s0_random.copy()

    # Combine into an 8x8 matrix
    P_random = np.block(
        [[P_s0_random, np.zeros((4, 4))], [np.zeros((4, 4)), P_s1_random]]
    )

    print("\nRandom 8x8 Transition Matrix P:\n", P_random)

    # --------------------------------------------------------------------
    # 4. Randomly define an initial distribution p(0)
    # --------------------------------------------------------------------
    # We'll separately sample probabilities for where each object is, then
    # form p(0) by multiplying. For example:
    p_s0_rand = np.random.rand()  # static in 0
    p_s1_rand = 1 - p_s0_rand  # static in 1
    p_i0_rand = np.random.rand()  # independent in 0
    p_i1_rand = 1 - p_i0_rand
    p_d0_rand = np.random.rand()  # dependent in 0
    p_d1_rand = 1 - p_d0_rand

    p0_random = np.zeros(8)
    # s=0 block
    p0_random[0] = p_s0_rand * p_i0_rand * p_d0_rand  # (0,0,0)
    p0_random[1] = p_s0_rand * p_i0_rand * p_d1_rand  # (0,0,1)
    p0_random[2] = p_s0_rand * p_i1_rand * p_d0_rand  # (0,1,0)
    p0_random[3] = p_s0_rand * p_i1_rand * p_d1_rand  # (0,1,1)
    # s=1 block
    p0_random[4] = p_s1_rand * p_i0_rand * p_d0_rand  # (1,0,0)
    p0_random[5] = p_s1_rand * p_i0_rand * p_d1_rand  # (1,0,1)
    p0_random[6] = p_s1_rand * p_i1_rand * p_d0_rand  # (1,1,0)
    p0_random[7] = p_s1_rand * p_i1_rand * p_d1_rand  # (1,1,1)

    # Normalize just in case rounding errors are big:
    p0_random /= p0_random.sum()

    print("\nRandom initial distribution p(0):", p0_random)
    print("Check sum:", p0_random.sum())

    # --------------------------------------------------------------------
    # 5. Convergence routine
    # --------------------------------------------------------------------
    def l1_distance(p1, p2):
        return np.sum(np.abs(p1 - p2))

    max_steps = 100000
    tolerance = 1e-3

    p_dist = p0_random.copy()
    for step in range(1, max_steps + 1):
        p_next = P_random.T @ p_dist
        dist = l1_distance(p_next, p_dist)
        p_dist = p_next

        if step < 6:  # print first few steps for illustration
            print(f"\nStep {step}: p={p_dist},  L1 dist={dist:.6g}")

        if dist < tolerance:
            print(f"\nConverged at step {step} (L1 distance={dist:.2e} < {tolerance})")
            step_needed_for_convergence.append(step)
            break

    if step == max_steps:
        print("\nReached max_steps without satisfying convergence tolerance.")

    print("\nFinal distribution:", p_dist)
    print("Check sum:", p_dist.sum())

print(
    f"The average number of steps needed for convergence is {np.mean(step_needed_for_convergence)}"
)
print(
    f"The standard deviation of the number of steps needed for convergence is {np.std(step_needed_for_convergence)}"
)

Random T_I:
 [[0.43418691 0.56581309]
 [0.52521691 0.47478309]]

Random attachment probabilities: a_0 = 0.4236547993389047 , a_1 = 0.6458941130666561

Random 8x8 Transition Matrix P:
 [[0.43418691 0.         0.32610366 0.23970943 0.         0.
  0.         0.        ]
 [0.         0.43418691 0.         0.56581309 0.         0.
  0.         0.        ]
 [0.52521691 0.         0.47478309 0.         0.         0.
  0.         0.        ]
 [0.33923451 0.1859824  0.         0.47478309 0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.43418691 0.
  0.32610366 0.23970943]
 [0.         0.         0.         0.         0.         0.43418691
  0.         0.56581309]
 [0.         0.         0.         0.         0.52521691 0.
  0.47478309 0.        ]
 [0.         0.         0.         0.         0.33923451 0.1859824
  0.         0.47478309]]

Random initial distribution p(0): [0.37604864 0.01417983 0.04563786 0.00172089 0.4833198  0.01822474
 0.05865646 0.0022